In [7]:
# Ollama 모델 로드 및 테스트
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="gemma:7b-instruct", temperature=0)

response = model.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")

In [8]:
response

AIMessage(content='겨울철에 내한성이 강한 나무는 다음과 같습니다.\n\n* **동백나무** : 동백나무는 겨울철에도 온도가 낮아지더라도 내한성이 강하여 손상을 입히기가 쉽지 않습니다.\n* **오동나무** : 오동나무는 동백나무와 마찬가지로 내한성이 강하여 손상을 입히기가 쉽지 않습니다.\n* **느린잎나무** : 느린잎나무는 겨울철에도 내한성이 강하여 손상을 입히기가 쉽지 않습니다.\n* **참나무** : 참나무는 겨울철에도 내한성이 강하여 손상을 입히기가 쉽지 않습니다.\n* **꽃나무** :꽃나무는 겨울철에도 내한성이 강하여 손상을 입히기가 쉽지 않습니다.', response_metadata={'model': 'gemma:7b-instruct', 'created_at': '2024-04-21T00:32:37.6390066Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 33737206500, 'load_duration': 7456666900, 'prompt_eval_count': 29, 'prompt_eval_duration': 1117902000, 'eval_count': 216, 'eval_duration': 25158538000}, id='run-65416b32-33e2-45a1-9d9f-0913149a436b-0')

In [1]:
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("sr650v3.pdf", extract_images=False)
pages = loader.load()
pages[0].page_content

'Lenovo ThinkSystem SR650 V3 Server\nProduct Guide\nThe Lenovo ThinkSystem SR650 V3 is an ideal 2-socket 2U rack server for small businesses up to large\nenterprises that need industry-leading reliability, management, and security, as well as maximizing performance\nand flexibility for future growth. The SR650 V3 is based on the new 5th generation Intel Xeon Scalable\nprocessor family (formerly codenamed "Emerald Rapids").\nThe SR650 V3 is designed to handle a wide range of workloads, such as databases, virtualization and cloud\ncomputing, virtual desktop infrastructure (VDI), infrastructure security, systems management, enterprise\napplications, collaboration/email, streaming media, web, and HPC.\nFigure 1. Lenovo ThinkSystem SR650 V3 with 2.5-inch front drive bays (3.5-inch drive configurations also\navailable)\nDid you know?\nThe SR650 V3 server has been designed to take advantage of the features of the 5th generation Intel Xeon\nScalable processors, such as the full performance of 

In [10]:
# 문장 임베딩 및 벡터 저장소 생성
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

print(torch.cuda.is_available())
# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

docs = text_splitter.split_documents(pages)

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)


True


In [23]:
# 검색 쿼리
query = "All Intel Xeon Silver CPU models list for SR650 V3 server"

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

docs = retriever.get_relevant_documents(query)
print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

4
Processors
The SR650 V3 supports processors in either the 5th Gen Intel Xeon Scalable Processor family or the 4th Gen
Intel Xeon Scalable Processor family. The server supports one or two processors.
Topics in this section:
Processor options
Processor features
Intel On Demand feature licensing
One-processor configurations
Thermal requirements for processors
Lenovo Neptune Processor DWC Module - Open-loop liquid cooling
UEFI operating modes
Processor options
All supported processors have the following characteristics:
8 DDR5 memory channels at 2 DIMMs per channel
Up to 4 UPI links between processors at up to 20 GT/s
80 PCIe 5.0 I/O lanes
The following table lists the 5th Gen processors that are currently supported by the SR650 V3.
Table 15. 5th Gen Intel Xeon Processor support
Part
number
Feature
code
SKU
Description
Quantity
supported
CTO only
BYVT
3508U
Intel Xeon Bronze 3508U 8C 125W 2.1GHz Processor
1*
4XG7A91456
BYW3
4509Y
ThinkSystem SR650 V3 Intel Xeon Silver 4509Y 8C 125W 2.6GH

In [21]:
print(docs[1].page_content)
print(docs[1].metadata)

Processors
The SR650 V3 supports processors in either the 5th Gen Intel Xeon Scalable Processor family or the 4th Gen
Intel Xeon Scalable Processor family. The server supports one or two processors.
Topics in this section:
Processor options
Processor features
Intel On Demand feature licensing
One-processor configurations
Thermal requirements for processors
Lenovo Neptune Processor DWC Module - Open-loop liquid cooling
UEFI operating modes
Processor options
All supported processors have the following characteristics:
8 DDR5 memory channels at 2 DIMMs per channel
Up to 4 UPI links between processors at up to 20 GT/s
80 PCIe 5.0 I/O lanes
The following table lists the 5th Gen processors that are currently supported by the SR650 V3.
Table 15. 5th Gen Intel Xeon Processor support
Part
number
Feature
code
SKU
Description
Quantity
supported
CTO only
BYVT
3508U
Intel Xeon Bronze 3508U 8C 125W 2.1GHz Processor
1*
4XG7A91456
BYW3
4509Y
ThinkSystem SR650 V3 Intel Xeon Silver 4509Y 8C 125W 2.6GHz


In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "4509Y CPU 2개 설치가 가능한가?"
rag_chain.invoke(query)


In [24]:
query = "겨울철에 추위에 약한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)

'겨울철에 추위에 약한 나무는 삼나무, 편백, 금송, 히말라야시다, 배롱나무, 파라칸타, 동백나무, 후박나무, 먼나무 등 주로 남부 지역에서 자라는 수종과 자목련, 사철나무, 가이즈까향나무, 능소화, 벽오동, 오동나무 등입니다.'